# Phase space MonteCarlo
Diffractive produciton of the $\eta\pi$-system:
$$
\pi^-+p\to X^- + p,\quad X^-\to \eta\pi^-, \quad \eta\to\pi^+\pi^-\pi^0[\to\gamma\gamma]
$$

### Function to boost, rotate and decay

In [31]:
function boost!(v,γ)
    β=sqrt(1-1/γ^2)
    v[1],v[4] = v[1]*γ+v[4]*β*γ, v[1]*β*γ+v[4]*γ
end

function rotateY!(v,cosθ)
    sinθ = sqrt(1-cosθ*cosθ)
    v[2],v[4] = v[2]*cosθ+v[4]*sinθ, -v[2]*sinθ+v[4]*cosθ
end

function rotateZ!(v,ϕ)
    v[2],v[3] = v[2]*cos(ϕ)-v[3]*sin(ϕ), v[3]*cos(ϕ)+v[2]*sin(ϕ)
end

λ(x,y,z) = x^2+y^2+z^2-2*x*y-2*y*z-2*z*x

function decay2two(v0,m1sq,m2sq)
    cosθ = 2*rand()-1;
    decay2two(v0,m1sq,m2sq,cosθ)
end

function decay2two(v0,m1sq,m2sq,cosθ)
    p0sq = v0[2]*v0[2]+v0[3]*v0[3]+v0[4]*v0[4];
    msq = v0[1]*v0[1]-p0sq
    (sqrt(msq) < sqrt(m1sq)+sqrt(m2sq)) && return [0,0,0,0], [0,0,0,0]
    p = sqrt(λ(msq,m1sq,m2sq)/(4*msq));
    ϕ = (2*rand()-1)*π;
    sinθ = sqrt(1-cosθ*cosθ);
    v1 = [(msq+m1sq-m2sq)/(2*sqrt(msq)),p*sinθ*sin(ϕ),p*sinθ*cos(ϕ),p*cosθ];
    v2 = [(msq-m1sq+m2sq)/(2*sqrt(msq)),-p*sinθ*sin(ϕ),-p*sinθ*cos(ϕ),-p*cosθ];
    # boost and rotate to the Lab frame
    γ0 = v0[1]/sqrt(msq)
    boost!(v1,γ0); boost!(v2,γ0); 
    # rotate appropriately
    if (p0sq > 0) 
        cosθ0 = v0[4]/sqrt(p0sq)
        rotateY!(v1,cosθ0); rotateY!(v2,cosθ0); 
    end
    ϕ0 = atan2(v0[3],v0[2])
    rotateZ!(v1,ϕ0); rotateZ!(v2,ϕ0);
    return v1,v2
end

decay2two (generic function with 2 methods)

In [32]:
v1,v2 = decay2two([1,0.1,0,0],0.1,0.1)
print(v1," ", v2,"\n")
print(masssq(v1)," ", masssq(v2))
v1+v2


[0.471723, -0.232774, 0.206747, 0.159983] [0.528277, 0.332774, -0.206747, -0.159983]
0.1000000000000001 0.09999999999999994

4-element Array{Float64,1}:
 1.0
 0.1
 0.0
 0.0

In [3]:
# set constants
const mπ = 0.139; const mπ2 = mπ^2;
const mη = 0.548; const mη2 = mη^2;
const mp = 0.938; const mp2 = mp^2;

# functions to generate random variables
fρ(s,m1sq,m2sq,m3sq,m0sq)=(sqrt(s)>sqrt(m1sq)+sqrt(m2sq)) ? sqrt(λ(s,m1sq,m2sq)*λ(m0sq,s,m3sq))/s : 0;

const dX = collect(linspace((mη+mπ)^2,3^2,100))
fX = [fρ(s,mη2,mπ2,mp2,19^2) for s in 0.5*(dX[1:end-1]+dX[2:end])]; fX /= sum(fX)
[fX[i] = fX[i-1]+fX[i] for i in 2:length(fX)];
const cX = fX;

const dππ = collect(linspace((mπ+mπ)^2,(mη-mπ)^2,100))
fππ = [fρ(s,mπ2,mπ2,mπ2,mη2) for s in 0.5*(dππ[1:end-1]+dππ[2:end])]; fππ /= sum(fππ)
[fππ[i] = fππ[i-1]+fππ[i] for i in 2:length(fππ)];
const cππ = fππ;

# tprime destribution
ρtp(tp) = tp*exp(-8.5*tp)
const dtp = collect(linspace(0.1,1,100))
ftp = [ρtp(tp) for tp in 0.5*(dtp[1:end-1]+dtp[2:end])]; ftp /= sum(ftp)
[ftp[i] = ftp[i-1]+ftp[i] for i in 2:length(ftp)];
const ctp = ftp;

In [21]:
# X disctribution
function randX()
    bi = searchsortedlast(cX,rand())+1
    return dX[bi] + rand()*(dX[bi+1]-dX[bi])
end
# ππ disctribution
function randPiPi()
    bi = searchsortedlast(cππ,rand())+1
    return dππ[bi] + rand()*(dππ[bi+1]-dππ[bi])
end
# tp disctribution
function randtp()
    bi = searchsortedlast(ctp,rand())+1
    return dtp[bi] + rand()*(dtp[bi+1]-dtp[bi])
end

randtp (generic function with 1 method)

In [6]:
using Plots

In [23]:
histogram([randX() for i=1:1000000],bins=100, xlab = "mX^2")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 12000 
 
 
 mX^2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [24]:
histogram([randPiPi() for i=1:1000000],bins=100, xlab = "mPiPi^2")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.08 
 
 
 0.10 
 
 
 0.12 
 
 
 0.14 
 
 
 0.16 
 
 
 0 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 10000 
 
 
 12500 
 
 
 mPiPi^2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [25]:
plot(x->ρtp(x),0.1,1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
<polyline clip-path="url(#clip2802)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.8117,15.2342 40.1541,14.2922 41.4965,13.5091 42.8389,12.8795 49.0451,11.811 55.2513,13.4159 61.4575,17.2591 67.6637,22.9594 80.076,38.6393 92.4884,58.269 
 107.407,84.7934 122.326,112.597 134.939,136.043 147.553,158.809 161.254,182.32 174.956,204.264 187.588,222.961 200.22,240.137 213.763,256.866 227.305,271.907 
 242.996,287.335 258.686,300.781 286.574,320.36 311.029,333.678 337.692,344.898 364.513,353.434 394.167,360.407 422.382,365.23 451.719,368.86 476.415,371.102 
 506.566,373.097 534.908,374.428 580.315,375.813 
 "/>
 
 
 
 
 y1

In [28]:
histogram([randtp() for i=1:1e6],bins=100, xlab = "tp^2")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 tp^2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [35]:
function generate()
    p0 = [190,0,0,sqrt(190*190-mπ2)] + [mp,0,0,0];
    s0 = p0[1]*p0[1]-p0[2]*p0[2]-p0[3]*p0[3]-p0[4]*p0[4];
    # generate variables
    sX = randX(); sPiPi = randPiPi(); tp=randtp();
    tmin = sX+mπ2-(s0+mπ2-mp2)*(s0+sX-mp2)/(2*s0)+sqrt(λ(s0,mπ2,mp2)*λ(s0,sX,mp2))/(2*s0);
    t = tmin-tp;
    cosθ = (2*s0*(t-sX-mπ2)+(s0+mπ2-mp2)*(s0+sX-mp2))/sqrt(λ(s0,mπ2,mp2)*λ(s0,sX,mp2))
    pX,pR = decay2two(p0,sX,mp2,cosθ)
    pη,pπm1 = decay2two(pX,mη2,mπ2)
    pππ,pπ0 = decay2two(pη,sPiPi,mπ2)
    pπm2,pπp = decay2two(pππ,mπ2,mπ2)
    pγ1,pγ2 = decay2two(pπ0,0,0)
    pπm1,pπm2,pπp,pγ1,pγ2,pR
end

generate (generic function with 1 method)

In [44]:
@time begin
histogram([begin
        pπm1,pπm2,pπp,pγ1,pγ2,pR = generate()
        sqrt(masssq(pπm1+pπm2+pπp+pγ1+pγ2))
                end for i=1:1e6], xlab = "Invariant mass of the etaPi system (GeV)")
end

  3.635298 seconds (93.03 M allocations: 3.294 GiB, 11.78% gc time)


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 0 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 10000 
 
 
 12500 
 
 
 Invariant mass of the etaPi system (GeV) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1